In [6]:
import requests
import pandas as pd

In [19]:
df = pd.read_csv('notebooks/claims.csv')
df

,Category,Question,Answer,Section
0,Manage existing benefit,How do I update my benefit information?,You can update your benefit information online...,general claim benefits
1,Manage existing benefit,What if my circumstances change?,Report changes in circumstances immediately to...,general claim benefits
2,Manage existing benefit,Can I appeal a decision?,Yes you can appeal within one month of the dec...,general claim benefits
3,Manage existing benefit,How can I report a change in income?,Report changes in income using the online portal.,general claim benefits
4,Manage existing benefit,How do I stop claiming benefits?,You can stop claiming by contacting your benef...,general claim benefits
...,...,...,...,...
420,Sodium Valproate Claims,What are the legal implications of prescribing...,Prescribing sodium valproate during pregnancy ...,nhs claim benefits
421,Sodium Valproate Claims,Can fathers of children affected by sodium val...,Yes fathers can also claim compensation if the...,nhs claim benefits
422,Sodium Valproate Claims,What role do support groups play in sodium val...,Support groups provide resources information a...,nhs claim benefits
423,Sodium Valproate Claims,Can you claim for future care needs due to sod...,Yes future care needs including specialized ed...,nhs claim benefits


In [21]:
docs_url = 'https://raw.githubusercontent.com/KayDD1/uk-benefits-assistant/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

In [16]:
df.columns = df.columns.str.lower()
df

,category,question,answer,section
0,Manage existing benefit,How do I update my benefit information?,You can update your benefit information online...,general claim benefits
1,Manage existing benefit,What if my circumstances change?,Report changes in circumstances immediately to...,general claim benefits
2,Manage existing benefit,Can I appeal a decision?,Yes you can appeal within one month of the dec...,general claim benefits
3,Manage existing benefit,How can I report a change in income?,Report changes in income using the online portal.,general claim benefits
4,Manage existing benefit,How do I stop claiming benefits?,You can stop claiming by contacting your benef...,general claim benefits
...,...,...,...,...
420,Sodium Valproate Claims,What are the legal implications of prescribing...,Prescribing sodium valproate during pregnancy ...,nhs claim benefits
421,Sodium Valproate Claims,Can fathers of children affected by sodium val...,Yes fathers can also claim compensation if the...,nhs claim benefits
422,Sodium Valproate Claims,What role do support groups play in sodium val...,Support groups provide resources information a...,nhs claim benefits
423,Sodium Valproate Claims,Can you claim for future care needs due to sod...,Yes future care needs including specialized ed...,nhs claim benefits


In [17]:
documents = df.to_dict(orient='records')

In [18]:
documents[0]

{'category': 'Manage existing benefit',
 'question': 'How do I update my benefit information?',
 'answer': 'You can update your benefit information online through your account.',
 'section': 'general claim benefits'}

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [ ]:
len(model.encode("Getting size of model dim"))

In [ ]:
sections = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["category"]).tolist()
    sections.append(doc)

In [ ]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "answer": {"type": "text"},
            "question": {"type": "text"},
            "category": {"type": "keyword"},
            "text_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
        }
    }
}
index_name = "uk-benefit-claims"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in sections:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [ ]:
search_term = "How do I stop claiming benefits?"
vector_search_term = model.encode(search_term)

In [ ]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [ ]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

In [ ]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}
response = es_client.search(
    index=index_name,
    query={
        "match": {"category": "Manage existing benefit"},
    },
    knn=knn_query,
    size=5
)
response["hits"]["hits"]